In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

#Imports
import glob
import functools
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import re
import numpy as np 
import pandas as pd

# Input data files are available in the read-only "../input/" directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/legalai/Object_casedocs'):
   for filename in filenames:
       print(os.path.join(dirname, filename))

## Data Handling
### We first wrap up all the text files into a single csv(comma separated file)

In [9]:
import glob
import csv
import os
import pandas as pd

# Define the dataset folder path correctly
base_path = r"D:\CODEing\Legal Assist\AILA-Artificial-Intelligence-for-Legal-Assistance-master\AILA-Artificial-Intelligence-for-Legal-Assistance-master\AILA_2019_dataset"

# Get all text files inside the Object_casedocs folder
read_files = glob.glob(os.path.join(base_path, "Object_casedocs", "*.txt"))

# Write all file contents into a CSV file (one file per row)
with open("object_casedocs.csv", "w", newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["filename", "content"])  # Optional header row
    for f in read_files:
        with open(f, "r", encoding="utf-8") as infile:
            content = " ".join([line.strip() for line in infile])
            writer.writerow([os.path.basename(f), content])

# Create DataFrame of just filenames
lst_arr = os.listdir(os.path.join(base_path, "D:\CODEing\Legal Assist\AILA_2019_dataset\Object_casedocs"))
df_filename = pd.DataFrame(lst_arr, columns=['Name'])
df_filename.head()


,Name
0,C1.txt
1,C10.txt
2,C100.txt
3,C1000.txt
4,C1001.txt


In [5]:
evaluate = pd.read_csv(r'D:\CODEing\Legal Assist\AILA_2019_dataset\relevance_judgments_priorcases.txt', delimiter = " ", header = None)
evaluate.columns = ["Query_Number", "Q0", "Document" ,"Relevance"]
evaluate=evaluate.drop(columns=["Q0"])
evaluate

,Query_Number,Document,Relevance
0,AILA_Q1,C168,0
1,AILA_Q1,C382,0
2,AILA_Q1,C428,0
3,AILA_Q1,C949,0
4,AILA_Q1,C2303,0
...,...,...,...
145695,AILA_Q50,C1367,0
145696,AILA_Q50,C2079,0
145697,AILA_Q50,C2066,0
145698,AILA_Q50,C1951,0


### A Glimpse about how the data inside the csv file looks!

In [6]:
df = pd.read_csv('D:\CODEing\Legal Assist\AILA_2019_dataset\object_casedocs.csv',header=None)
df.columns = ["Text"]
df

,Text
0,Gangadhar Behera and others v State of Orissa ...
1,Mangesh v State Of Maharashtra Supreme Court o...
2,East India Tobacco Company v State of Andhra P...
3,Mahavir Prashad Gupta and Another v State of N...
4,Uptron India Limited v Shammi Bhan and Another...
...,...
2909,Sangappa and Others v State of Karnataka Supre...
2910,Ramdas v State Of Madhya Pradesh Supreme Court...
2911,Ashok Dhingra v N.C.T. of Delhi Supreme Court ...
2912,Prafulla Kumar Das and Others with Pradip Chan...


In [10]:
df = pd.concat([df_filename,df], axis = 1)
df

,Name,Text
0,C1.txt,Gangadhar Behera and others v State of Orissa ...
1,C10.txt,Mangesh v State Of Maharashtra Supreme Court o...
2,C100.txt,East India Tobacco Company v State of Andhra P...
3,C1000.txt,Mahavir Prashad Gupta and Another v State of N...
4,C1001.txt,Uptron India Limited v Shammi Bhan and Another...
...,...,...
2909,C995.txt,Sangappa and Others v State of Karnataka Supre...
2910,C996.txt,Ramdas v State Of Madhya Pradesh Supreme Court...
2911,C997.txt,Ashok Dhingra v N.C.T. of Delhi Supreme Court ...
2912,C998.txt,Prafulla Kumar Das and Others with Pradip Chan...


### Let us get some basic information about the data

In [11]:
len(df)

2914

In [12]:
df.shape

(2914, 2)

In [13]:
df.info

<bound method DataFrame.info of            Name                                               Text
0        C1.txt  Gangadhar Behera and others v State of Orissa ...
1       C10.txt  Mangesh v State Of Maharashtra Supreme Court o...
2      C100.txt  East India Tobacco Company v State of Andhra P...
3     C1000.txt  Mahavir Prashad Gupta and Another v State of N...
4     C1001.txt  Uptron India Limited v Shammi Bhan and Another...
...         ...                                                ...
2909   C995.txt  Sangappa and Others v State of Karnataka Supre...
2910   C996.txt  Ramdas v State Of Madhya Pradesh Supreme Court...
2911   C997.txt  Ashok Dhingra v N.C.T. of Delhi Supreme Court ...
2912   C998.txt  Prafulla Kumar Das and Others with Pradip Chan...
2913   C999.txt  M. K. Gopalan and Another v State of Madhya Pr...

[2914 rows x 2 columns]>

## Text preprocessing techniques: Cleansing the data
### 1. Convert to lowercase, remove punctuation and special characters, using RegeX and strip
### 2. Remove stopwords
### 3. Stemming 
### 4. Lemmatization

In [14]:
import re
#Convert lowercase remove punctuation and Character and then strip 
text = df.iloc[0]
print(text)
text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
txt = text.split()
print(txt)

Name                                               C1.txt
Text    Gangadhar Behera and others v State of Orissa ...
Name: 0, dtype: object
['name', 'c1txt', 'text', 'gangadhar', 'behera', 'and', 'others', 'v', 'state', 'of', 'orissa', 'name', '0', 'dtype', 'object']


In [15]:
#remove stopwords
import nltk
lst_stopwords = nltk.corpus.stopwords.words("english")
txt = [word for word in txt if word not in lst_stopwords]
print(txt)

['name', 'c1txt', 'text', 'gangadhar', 'behera', 'others', 'v', 'state', 'orissa', 'name', '0', 'dtype', 'object']


In [16]:
#stemming
ps = nltk.stem.porter.PorterStemmer()
print([ps.stem(word) for word in txt])

['name', 'c1txt', 'text', 'gangadhar', 'behera', 'other', 'v', 'state', 'orissa', 'name', '0', 'dtype', 'object']


In [17]:
#Lemmetization
nltk.download('wordnet')
lem = nltk.stem.wordnet.WordNetLemmatizer()
print([lem.lemmatize(word) for word in txt])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cs594\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['name', 'c1txt', 'text', 'gangadhar', 'behera', 'others', 'v', 'state', 'orissa', 'name', '0', 'dtype', 'object']


## Preprocessing the data: Apply these techniques on all records of the dataset

In [18]:
#to apply all the technique to all the records on dataset
def utils_preprocess_text(text, flg_stemm=True, flg_lemm =True, lst_stopwords=None ):
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
    #tokenization(convert from string to List)
    lst_text = text.split()
    
    #remove stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in
                   lst_stopwords]
        
     #stemming
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
        
    #Lemmentization
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
        
    # back to string from list
    text = " ".join(lst_text)
    return text

df['clean_text'] = df['Text'].apply(lambda x: utils_preprocess_text(x, flg_stemm = False, flg_lemm=True))

## A Glimpse into the cleansed data!

In [19]:
#df
df

,Name,Text,clean_text
0,C1.txt,Gangadhar Behera and others v State of Orissa ...,gangadhar behera and others v state of orissa ...
1,C10.txt,Mangesh v State Of Maharashtra Supreme Court o...,mangesh v state of maharashtra supreme court o...
2,C100.txt,East India Tobacco Company v State of Andhra P...,east india tobacco company v state of andhra p...
3,C1000.txt,Mahavir Prashad Gupta and Another v State of N...,mahavir prashad gupta and another v state of n...
4,C1001.txt,Uptron India Limited v Shammi Bhan and Another...,uptron india limited v shammi bhan and another...
...,...,...,...
2909,C995.txt,Sangappa and Others v State of Karnataka Supre...,sangappa and others v state of karnataka supre...
2910,C996.txt,Ramdas v State Of Madhya Pradesh Supreme Court...,ramdas v state of madhya pradesh supreme court...
2911,C997.txt,Ashok Dhingra v N.C.T. of Delhi Supreme Court ...,ashok dhingra v nct of delhi supreme court of ...
2912,C998.txt,Prafulla Kumar Das and Others with Pradip Chan...,prafulla kumar da and others with pradip chand...


In [20]:
train = df["clean_text"]
train

0       gangadhar behera and others v state of orissa ...
1       mangesh v state of maharashtra supreme court o...
2       east india tobacco company v state of andhra p...
3       mahavir prashad gupta and another v state of n...
4       uptron india limited v shammi bhan and another...
                              ...                        
2909    sangappa and others v state of karnataka supre...
2910    ramdas v state of madhya pradesh supreme court...
2911    ashok dhingra v nct of delhi supreme court of ...
2912    prafulla kumar da and others with pradip chand...
2913    m k gopalan and another v state of madhya prad...
Name: clean_text, Length: 2914, dtype: object

In [21]:
import texthero as hero

### Creating a test dataframe using the Query file

In [22]:
test = pd.read_csv("D:\CODEing\Legal Assist\AILA_2019_dataset\Query_doc.txt",delimiter = "|",header=None)
test.columns = ["AILA","NAN", "Query"]
test=test.drop(columns=["AILA","NAN"])

In [23]:
test

,Query
0,"The appellant on February 9, 1961 was appointe..."
1,The appellant before us was examined as prime ...
2,This appeal arises from the judgment of the le...
3,The Petitioner was married to the Respondent N...
4,This appeal is preferred against the judgment ...
5,"On 19.3.1999, SI P1 along Ct. P2 went to Villa..."
6,This criminal appeal is directed against the j...
7,"This appeal, by special leave, has been prefer..."
8,The complainant P1 filed a Special Leave Petit...
9,"The four appellants, along with P1 son of P2, ..."


## Cleanse the test data
### We use the same methods as above to cleanse the test data

In [24]:
test['Query_processed'] = test['Query'].apply(lambda x: utils_preprocess_text(x, flg_stemm = False, flg_lemm=True))

In [25]:
test

,Query,Query_processed
0,"The appellant on February 9, 1961 was appointe...",the appellant on february 9 1961 wa appointed ...
1,The appellant before us was examined as prime ...,the appellant before u wa examined a prime wit...
2,This appeal arises from the judgment of the le...,this appeal arises from the judgment of the le...
3,The Petitioner was married to the Respondent N...,the petitioner wa married to the respondent no...
4,This appeal is preferred against the judgment ...,this appeal is preferred against the judgment ...
5,"On 19.3.1999, SI P1 along Ct. P2 went to Villa...",on 1931999 si p1 along ct p2 went to village v...
6,This criminal appeal is directed against the j...,this criminal appeal is directed against the j...
7,"This appeal, by special leave, has been prefer...",this appeal by special leave ha been preferred...
8,The complainant P1 filed a Special Leave Petit...,the complainant p1 filed a special leave petit...
9,"The four appellants, along with P1 son of P2, ...",the four appellant along with p1 son of p2 wer...


# BM-25 ranking

In [26]:
%pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.


In [27]:
from rank_bm25 import BM25Okapi

query_array_processed = [0]*50

corpus_array_processed = [0]*2914

train_array=df.iloc[:,1:].values

for i in range(2914):
    corpus_array_processed[i] = train_array[i][0]

query_array=test.iloc[:,1:].values

#test["Query_processed"]
#test.values(columns=[test["Query_processed"]])
#query_array[49][0]

for i in range(50):
    query_array_processed[i] = query_array[i][0]

In [28]:
train_array=df.iloc[:,1:].values
tokenized_corpus = [doc.split(" ") for doc in corpus_array_processed]

In [29]:
bm25 = BM25Okapi(tokenized_corpus)
bm25

In [30]:
name = df["Name"]
name = name.str.rstrip('.txt')
name

0          C1
1         C10
2        C100
3       C1000
4       C1001
        ...  
2909     C995
2910     C996
2911     C997
2912     C998
2913     C999
Name: Name, Length: 2914, dtype: object

In [31]:
bm25.get_top_n(corpus_array_processed[4].split(" "), name, n=10)

['C1001',
 'C518',
 'C186',
 'C673',
 'C1006',
 'C28',
 'C1340',
 'C689',
 'C1571',
 'C1847']

In [32]:
evaluate = evaluate.loc[evaluate['Relevance'] == 1]
evaluate

,Query_Number,Document,Relevance
1192,AILA_Q1,C14,1
2274,AILA_Q1,C9,1
3076,AILA_Q2,C27,1
3676,AILA_Q2,C22,1
6033,AILA_Q3,C1,1
...,...,...,...
140861,AILA_Q49,C38,1
142203,AILA_Q49,C76,1
142450,AILA_Q49,C92,1
143069,AILA_Q50,C27,1


In [33]:
retrieved = bm25.get_top_n(query_array_processed[i].split(" "), name, n=10)
relevant = evaluate.loc[evaluate['Query_Number'] == "AILA_Q"+str(i+1)]["Document"]

In [34]:
count = 0
for i in range(50):
    for j in retrieved:
        for k in relevant:
            if (j==k):
                count=count+1

print(count)

0


In [36]:
Precision = count/500
Recall = count/195

print(Precision)
print(Recall)

0.0
0.0
